# All Ireland M.bovis strain diversity and clade identification

In [1]:
import sys,os,io,shutil,subprocess
import glob
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
import matplotlib as mpl
from Bio import SeqIO
import seaborn as sns
sys.path.append('pathogenie')
from snipgenie import tools, aligners, app, trees, plotting
import toytree, toyplot
import toyplot.pdf
path = '/storage/btbgenie/all_ireland_results'

## get meta data

In [14]:
meta = pd.read_csv('/storage/btbgenie/mbovis_ireland/all_ireland_samples.csv')
print (len(meta))

583


In [ ]:
dups=meta[meta.duplicated('aliquot')].dropna()
#print (dups)
for i,r in dups.iterrows():
    print (meta[meta.aliquot== r.aliquot])
#meta[(meta.batch.isin([2,3,4])) & (~meta.aliquot.isin(dups.aliquot)) & (meta.county=='Monaghan')]

## run NI

In [ ]:
reload(app)
args = {'threads':20, 'outdir': '/storage/btbgenie/NI_results', 'labelsep':'_',
        'input':['/storage/btbgenie/mbovis_ireland/NI'],
         'species': 'Mbovis-AF212297',
         'overwrite':False,
         'custom_filters': True,       
         'buildtree':True}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

## misc 2021 samples

In [ ]:
args = {'threads':20, 'outdir': '/storage/btbgenie/misc_results', 'labelsep':'_',
        'input':['/storage/btbgenie/mbovis_ireland/Misc'],
         'species': 'Mbovis-AF212297',
         'overwrite':False,
         'custom_filters': True,
         'buildtree':True}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

In [ ]:
reload(app)
mask='/home/farrell/gitprojects/snipgenie/snipgenie/data/Mbovis_AF212297_mask.bed'
vcf_file='/storage/btbgenie/misc_results/snps.vcf.gz'
app.mask_filter(vcf_file, mask)

In [244]:
present=pd.read_csv('/storage/btbgenie/misc_results/summary.csv')
present['sample']=present['sample'].astype(str)
#print (present[:10])
l=list(W.fastq_table['sample'].unique())

s=meta[meta.batch>=3].sort_values(by='sample')
#s['label'] = s['sample'].apply(lambda x : x.split('-')[1][-4:]).astype(str)
#s=s.sort_values('label')

p = present[~present['sample'].isin(s['sample'])]
#p = (s[s['label'].isin(l)])
print (p)
print (len(s),len(p), len(present))

         sample                           name                                           bam_file  read_length
25  TB19-004280  TB19-004280_S68_L001-4_R2_001  /storage/btbgenie/misc_results/mapped/TB19-004...          149
48  TB19-005887  TB19-005887_S29_L001-4_R2_001  /storage/btbgenie/misc_results/mapped/TB19-005...          150
65 2 61


In [284]:
reload(trees)
spo=pd.read_csv('/storage/btbgenie/all_ireland_results/misc_spoligotypes.csv')
treefile = '/storage/btbgenie/misc_results/tree.newick'
tre = toytree.tree(treefile)
tipnames = tre.get_tip_labels()
mapping = dict(zip(meta['sample'],meta.county))
colormap = trees.colors_from_labels(meta,'name','county')
tip_colors = [colormap[mapping[i]] if i in mapping else 'Black' for i in tipnames]

canvas,t,r=tre.draw(layout='r',width=1000,height=1000,tip_labels_colors=tip_colors,
                    scalebar=True,);
toyplot.pdf.render(canvas,'/storage/btbgenie/all_ireland_results/misc_samples_tree.pdf')

## run CVRL samples

In [ ]:
args = {'threads':20, 'outdir': '/storage/btbgenie/cvrl_results', 'labelsep':'_',
        'input':['/storage/btbgenie/mbovis_ireland/CVRL'],
         'species': 'Mbovis-AF212297',
         'overwrite':False,
         'custom_filters': True,
         'buildtree':True}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

In [ ]:
reload(trees)
spo=pd.read_csv('/storage/btbgenie/all_ireland_results/cvrl_spoligotypes.csv')
treefile = '/storage/btbgenie/cvrl_results/tree.newick'
tre = toytree.tree(treefile)
tipnames = tre.get_tip_labels()
mapping = dict(zip(meta['sample'],meta.county))
colormap = trees.colors_from_labels(meta,'name','county')
tip_colors = [colormap[mapping[i]] if i in mapping else 'Black' for i in tipnames]

canvas,t,r=tre.draw(layout='r',width=1000,height=1400,tip_labels_colors=tip_colors,
                    scalebar=True,);

## integrate previously aligned bam files together and call variants

This allows us to join previously mapped results together in one call without re-aligning

In [41]:
#read in bam files
mapped = {'wicklow':'/storage/btbgenie/wicklow_results/mapped/',
          'cvrl':'/storage/btbgenie/cvrl_results/mapped/',
          'misc':'/storage/btbgenie/misc_results/mapped/',
          'monaghan':'/storage/btbgenie/monaghan/monaghan_results/mapped/',
          'NI':'/storage/btbgenie/NI_results/','UK':'/storage/btbgenie/UK_results/mapped'}
reload(app)
bam_files=[]
for m in mapped:
    f=app.get_files_from_paths(mapped[m], '*.bam')
    bam_files.extend(f)
print (len(bam_files))

outdir='/storage/btbgenie/all_ireland_results'
df = app.get_samples(bam_files,sep='_')
app.write_samples(df, outdir)

531


'/storage/btbgenie/all_ireland_results/samples.txt'

## run previously aligned files

In [42]:
gff_file=os.path.join(outdir,'mbovis.gff')
tools.gff_bcftools_format(app.mbovis_gb, gff_file)
app.run_bamfiles(bam_files, app.mbovis_genome, outdir='/storage/btbgenie/all_ireland_results', threads=20,
                 mask=app.mbovis_mask, gff_file=gff_file,
                 custom_filters=True, overwrite=False)

[      1  217496  434991  652486  869981 1087476 1304971 1522467 1739962
 1957457 2174952 2392447 2609942 2827437 3044933 3262428 3479923 3697418
 3914913 4132408 4349904]
parallel bcftools mpileup -r {1} -a "AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR" -O b --min-MQ 10 -o {2} -f /home/farrell/.config/snipgenie/genome/Mbovis_AF212297.fa /storage/btbgenie/wicklow_results/mapped/31-12952.bam /storage/btbgenie/wicklow_results/mapped/48-MBovis.bam /storage/btbgenie/wicklow_results/mapped/49-MBovis.bam /storage/btbgenie/wicklow_results/mapped/28-12935.bam /storage/btbgenie/wicklow_results/mapped/27-MBovis.bam /storage/btbgenie/wicklow_results/mapped/15-11643.bam /storage/btbgenie/wicklow_results/mapped/59-MBovis.bam /storage/btbgenie/wicklow_results/mapped/182-MBovis.bam /storage/btbgenie/wicklow_results/mapped/59-6110.bam /storage/btbgenie/wicklow_results/mapped/41-2165.bam /storage/btbgenie/wicklow_results/mapped/37-MBovis.bam /storage/btbgenie/wicklow_results/mapped/5-10284.bam /storage/b

## spoligotypes of all samples

In [213]:
paths = {'Wicklow':'/storage/btbgenie/mbovis_ireland/Wicklow','Monaghan':'/storage/btbgenie/mbovis_ireland/Monaghan/',
         'cvrl':'/storage/btbgenie/mbovis_ireland/CVRL',
         'NI':'/storage/btbgenie/mbovis_ireland/NI/'}
samples = []
#paths = {'misc':'/storage/btbgenie/mbovis_ireland/Jun21/'}
for p in paths:
    f = app.get_files_from_paths(paths[p])   
    df = app.get_samples(f,sep='_')
    print (len(df['sample'].unique()))
    df=df.drop_duplicates('sample')   
    df['county'] = p   
    samples.append(df)

samples = pd.concat(samples)

67


In [ ]:
reload(tools)
res=[]
for i,r in samples.iterrows():
    f=r.filename
    samp=r['sample']
    b = tools.get_spoligotype(f,1e6)
    sb = tools.get_sb_number(b)
    print (f,samp,sb, b)
    res.append([f,samp,sb,b])
    
res = pd.DataFrame(res,columns=['filename','sample','SB','code'])
res.code.astype(object)
#res.to_csv('/storage/btbgenie/all_ireland_results/spoligotypes.csv',index=False)

In [215]:
#res.to_csv('/storage/btbgenie/all_ireland_results/misc_spoligotypes.csv',index=False)

## clusters

In [12]:
clust = trees.run_tree_cluster('/storage/btbgenie/all_ireland_results/tree.newick',dist=10)
#clust = pd.read_csv('monaghan_results/clusters.txt',sep='\t')
clust.ClusterNumber.value_counts()


TreeCluster.py  -i /storage/btbgenie/all_ireland_results/tree.newick -t 10


-1     39
 52    39
 11    35
 26    33
 21    28
       ..
 29     2
 30     2
 32     2
 33     2
 68     2
Name: ClusterNumber, Length: 69, dtype: int64

In [ ]:
meta.county.value_counts()

In [11]:
spo = pd.read_csv('/storage/btbgenie/all_ireland_results/spoligotypes.csv')
#clust = pd.read_csv('/storage/btbgenie/all_ireland_results/clusters.txt',sep='\t')

In [15]:
final = meta.merge(spo,on='sample',how='left')
final = final.drop('code',1)
final = final.merge(clust,left_on='sample',right_on='SequenceName')
len(final)

530

## assign clade names?

In [31]:
clmap={}
for county,g in final.groupby('county'):
    c = g.value_counts('ClusterNumber')
    c = c[c>8]
    n=1
    for i,r in c.iteritems():
        if i==-1:
            continue
        clmap[i]=county+'-'+str(n)
        n+=1
clmap

{52: 'Clare-1',
 53: 'Clare-2',
 31: 'Clare-3',
 38: 'Clare-4',
 48: 'Clare-5',
 66: 'Clare-6',
 11: 'Monaghan-1',
 58: 'Monaghan-2',
 21: 'Monaghan-3',
 61: 'Monaghan-4',
 36: 'Monaghan-5',
 50: 'Monaghan-6',
 26: 'NI-1',
 46: 'Wicklow-1',
 27: 'Wicklow-2'}

In [29]:
final['clade'] = final.ClusterNumber.map(clmap)

In [30]:
final.to_csv('/storage/btbgenie/mbovis_ireland/ireland_metadata.csv',index=False)

## draw all ireland tree

In [44]:
treefile = '/storage/btbgenie/all_ireland_results/tree.newick'
tre = toytree.tree(treefile)
#canvas,t,r=tre.draw(layout='c')

In [45]:
reload(trees)
#tre = tre.root('TB20-002800')
#tre = tre.drop_tips('8567')
tipnames = tre.get_tip_labels()
mapping = dict(zip(final['sample'],final.clade))
#mapping['ref'] = 'AF2122/97'
tiplabels = [mapping[i] if i in mapping else 'NA' for i in tipnames]

mapping = dict(zip(meta['sample'],meta.county))
colormap = trees.colors_from_labels(meta,'name','county')
print (colormap)
tip_colors = [colormap[mapping[i]] if i in mapping else 'Black' for i in tipnames]
tip_labels_style={      
        "font-size": "6px",
        "-toyplot-anchor-shift": "13px",
    }
node_sizes=[0 if i else 5 for i in tre.get_node_values(None, 1, 0)]
node_colors = [colormap[mapping[n]] if n in mapping else 'black' for n in tre.get_node_values('name', True, True)]

canvas,t,r=tre.draw(layout='r',width=1200,height=1300,node_markers="o",node_hover=True, node_sizes=node_sizes,
                    tip_labels=False,tip_labels_style=tip_labels_style,node_colors=node_colors,
                    tip_labels_colors=tip_colors,scalebar=True,edge_widths=1);
#toyplot.pdf.render(canvas,'/storage/btbgenie/all_ireland_results/ireland_tree.pdf')

{'NI': 'blue', 'Wicklow': 'green', 'Monaghan': 'crimson', 'Clare': 'blueviolet', 'Cavan': 'orange', 'Longford': 'cadetblue', 'Cork': 'chartreuse', 'Louth': 'chocolate', 'Kilkenny': 'coral', 'Meath': 'gold', 'Wexford': 'cornflowerblue', 'Mayo': 'palegreen', 'Kildare': 'khaki', 'Laois': 'orange', 'Roscommon': 'pink', 'Donegal': 'burlywood', 'Offaly': 'red', 'Galway': 'lime', 'Tipperary': 'mediumvioletred', 'Westmeath': 'navy', 'Sligo': 'teal', 'Kerry': 'darkblue', 'Leitrim': 'purple', 'Limerick': 'orange', 'Waterford': 'salmon', 'Dublin': 'maroon', nan: 'blue'}


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1200.0px" height="1300.0px" viewBox="0 0 1200.0 1300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1963fc3c5220415681fd57c02b20a82a"> idx: 0
dist: 2.0016
support: 0.0000
height: 36.8555
name: TB19-006247 idx: 1
dist: 3.0033
support: 0.0000
height: 35.8538
name: TB19-006257 idx: 2
dist: 1.0008
support: 0.0000
height: 35.8600
name: TB19-003397 idx: 3
dist: 1.0008
support: 0.0000
height: 35.8600
name: TB19-006168 idx: 4
dist: 13.0497
support: 0.0000
height: 32.8295
name: TB19-005804 idx: 5
dist: 11.0402
support: 0.0000
height: 37.8593
name: TB20-005008 idx: 6
dist: 11.0453
support: 0.0000
height: 37.8542
name: TB19-004843 idx: 7
dist: 2.0319
support: 0.0000
height: 29.8537
name: TB19-004646 idx: 8
dist: 2.9663
support: 0.0000
height: 28.9193
name: TB20-001041 idx: 9
dist: 1.0321
support: 0.0000
height: 31.8536
name: TB19-003885 idx: 10
dist: 7.0065
support: 0.0000
height: 36.8637
name: TB19-000708 idx: 11
dist: 4.0042
support: 0.0000
height: 38.8721
name: TB19-005144 idx: 12
dist: 6.0060
support: 0.0000
height: 36.8703
name: TB20-007935 idx: 13
dist: 0.9999
support: 0.0000
height: 36.8600
name: TB20-013234 idx: 14
dist: 0.9997
support: 0.0000
height: 36.8602
name: TB20-007117 idx: 15
dist: 0.9997
support: 0.0000
height: 36.8650
name: TB20-011673 idx: 16
dist: 2.0017
support: 0.0000
height: 37.8681
name: TB19-000499 idx: 17
dist: 0.0048
support: 0.0000
height: 39.8650
name: TB19-001116 idx: 18
dist: 11.0380
support: 0.0000
height: 36.8450
name: TB20-012859 idx: 19
dist: 2.0011
support: 0.0000
height: 34.8314
name: TB20-002987 idx: 20
dist: 0.0048
support: 0.0000
height: 36.8278
name: TB19-011618 idx: 21
dist: 0.0048
support: 0.0000
height: 31.8210
name: TB19-011395 idx: 22
dist: 0.0048
support: 0.0000
height: 31.8210
name: TB19-008959 idx: 23
dist: 1.0003
support: 0.0000
height: 30.8302
name: TB20-011697 idx: 24
dist: 2.0011
support: 0.0000
height: 29.8341
name: TB19-011445 idx: 25
dist: 3.0028
support: 0.0000
height: 37.8559
name: TB19-000031 idx: 26
dist: 0.0048
support: 0.0000
height: 38.8494
name: TB20-001637 idx: 27
dist: 0.0048
support: 0.0000
height: 38.8494
name: TB19-003858 idx: 28
dist: 14.0720
support: 0.0000
height: 30.8172
name: TB20-007429 idx: 29
dist: 9.0327
support: 0.0000
height: 35.8565
name: TB19-004622 idx: 30
dist: 3.0043
support: 0.0000
height: 44.8823
name: TB20-001761 idx: 31
dist: 17.0775
support: 0.0000
height: 31.8050
name: TB19-005229 idx: 32
dist: 7.0167
support: 0.0000
height: 31.8366
name: TB19-004984 idx: 33
dist: 4.0038
support: 0.0000
height: 34.8496
name: TB19-008302 idx: 34
dist: 10.0307
support: 0.0000
height: 34.8533
name: 19-4281 idx: 35
dist: 6.0095
support: 0.0000
height: 38.8745
name: TB19-004037 idx: 36
dist: 23.1184
support: 0.0000
height: 26.7713
name: TB20-003818 idx: 37
dist: 21.1136
support: 0.0000
height: 33.7880
name: TB19-001492 idx: 38
dist: 0.0048
support: 0.0000
height: 29.7955
name: TB19-008949 idx: 39
dist: 4.0072
support: 0.0000
height: 25.7930
name: TB19-008952 idx: 40
dist: 1.0015
support: 0.0000
height: 31.8039
name: TB19-008951 idx: 41
dist: 1.0016
support: 0.0000
height: 31.8039
name: TB19-006006 idx: 42
dist: 1.0016
support: 0.0000
height: 31.8038
name: TB19-006007 idx: 43
dist: 1.0014
support: 0.0000
height: 31.8136
name: TB19-005970 idx: 44
dist: 6.0097
support: 0.0000
height: 42.8729
name: TB20-009522 idx: 45
dist: 6.0350
support: 0.0000
height: 38.8572
name: TB19-007739 idx: 46
dist: 9.0333
support: 0.0000
height: 35.8589
name: TB19-003881 idx: 47
dist: 12.0375
support: 0.0000
height: 33.8423
name: TB19-005570 idx: 48
dist: 2.0026
support: 0.0000
height: 

## SNP analysis

* snps unique to each clade/county
* protein groups containing common snps

In [35]:
csq = app.read_csq_file('/storage/btbgenie/all_ireland_results/csq.tsv')
snp_mat = app.get_aa_snp_matrix(csq)
#csq_ind = app.read_csq_file('/storage/btbgenie/all_ireland_results/csq_indels.tsv')
#ind_mat = app.get_aa_snp_matrix(csq_ind)

In [36]:
mat = snp_mat

#labelmap = dict(zip(meta['name'],meta.county))
#mat = mat.rename(columns=labelmap)
#freqs = mat.sum(1)/len(mat.columns)
#freqs =freqs.rename('freq') 
#freqs.sort_values()
#freqs.hist(bins=20)

In [37]:
#mat.index = mat.index.get_level_values(1)+'_'+mat.index.get_level_values(2)
stat = mat.sum(1).reset_index()
#stat.gene.value_counts()[:30]

In [ ]:
cols=['sample','county','SB','ClusterNumber','clade']
C=mat.T
C=C.merge(meta[cols],left_index=True,right_on='sample').set_index(cols).T
#print (C)
C=C[C.sum(1)>3].T
C.iloc[:4,:2]
C.to_csv('csq_snps_ireland.csv')

In [257]:
counties = list(C.index.get_level_values(1))
sb = list(C.index.get_level_values(2))
clust = C.index.get_level_values(3)
clades = C.index.get_level_values(4)

In [ ]:
colormap1 = dict(zip(clades.unique(), trees.qcolors))
#colormap1 = {'Wicklow-1': 'green', 'Monaghan-3': 'red', np.nan: 'gray', 
#             'Monaghan-1': 'brown', 'Monaghan-2': 'burlywood', 'Unknown': 'lightblue', 'NI-1': 'blue'}
colors1 = [colormap1[i] if i in colormap1 else 'Black' for i in clades]
#make legend
import matplotlib.patches as mpatches
pts=[]
for c in colormap1:
    pts.append(mpatches.Patch(color=colormap1[c],label=c))
g=sns.clustermap(C,xticklabels=False,yticklabels=False,figsize=(17,12),cmap='gray_r',cbar_pos=None,row_colors=colors1)
g.fig.legend(handles=pts,bbox_to_anchor=(1.05, 1), )
g.savefig('/storage/btbgenie/all_ireland_results/snp_clusters.jpg')

## PCA

In [33]:
import sklearn
from sklearn import decomposition
from sklearn import manifold

pca = decomposition.PCA(n_components=3)
mds = manifold.MDS(n_components=3)
pos = mds.fit(C).embedding_
X = pca.fit_transform(pos)
#X = pca.transform(C)
df=pd.DataFrame(X)
df['sb']=sb
df.to_csv('/storage/btbgenie/all_ireland_results/pca_snp.csv')

fig,ax = plt.subplots(figsize=(7, 7))
sc=ax.scatter(X[:, 0], X[:, 1], c=colors1, s=100, alpha=.6)    
ax.legend(handles=pts,loc='best')
fig.savefig('/storage/btbgenie/all_ireland_results/pca_snp_clusters.jpg',dpi=150)

NameError: name 'C' is not defined

## compare cvrl wgmlst tree

In [351]:
cvrltre = toytree.tree('declan.newick')
cvrltre.draw(layout='c',width=800,height=1000,
                node_markers="o",node_hover=True, #node_sizes=node_sizes,
                tip_labels=False,tip_labels_style=tip_labels_style,#node_colors=node_colors,
                scalebar=True,edge_widths=1)

(<toyplot.canvas.Canvas at 0x7f1032dd6d00>,
 <toytree.Render.ToytreeMark at 0x7f1032389550>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1ac6205ac4f243e3a41c08510318b498"> -200 -100 0 100 200 -200 -100 0 100 200